In [31]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [32]:
odds_f = pd.read_csv(r"E:\project\nba_odds_study\data\odds\bettingdata_com\processed\bettingdata_odds.csv", parse_dates=["datetime"], index_col="datetime")

In [33]:
pre_f = pd.read_csv(r"E:\project\nba_odds_study\data\predictions\machine_learning\prediction_corrected.csv")

In [34]:
odds_f.columns

Index(['Season', 'Type', 'DayString', 'TimeString', 'DayOfWeek',
       'DateTimeString', 'LastUpdated', 'Status', 'AwayTeam', 'AwayTeamScore',
       ...
       'BarstoolAwayPointSpread', 'BarstoolHomePointSpreadPayout',
       'BarstoolAwayPointSpreadPayout', 'BarstoolOverUnder',
       'BarstoolOverPayout', 'BarstoolUnderPayout', 'BarstoolHomeTeamTotal',
       'BarstoolAwayTeamTotal', 'BarstoolHomeTeamTotalPayout',
       'BarstoolAwayTeamTotalPayout'],
      dtype='object', length=143)

In [35]:
pre_f.columns

Index(['Unnamed: 0', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA',
       'FG_PCT',
       ...
       'BLK_RANK.1', 'BLKA_RANK.1', 'PF_RANK.1', 'PFD_RANK.1', 'PTS_RANK.1',
       'PLUS_MINUS_RANK.1', 'Home-Team-Win', 'date', 'winprob_home',
       'winprob_away'],
      dtype='object', length=112)

In [36]:
odds_f.head(5)

,Season,Type,DayString,TimeString,DayOfWeek,DateTimeString,LastUpdated,Status,AwayTeam,AwayTeamScore,...,BarstoolAwayPointSpread,BarstoolHomePointSpreadPayout,BarstoolAwayPointSpreadPayout,BarstoolOverUnder,BarstoolOverPayout,BarstoolUnderPayout,BarstoolHomeTeamTotal,BarstoolAwayTeamTotal,BarstoolHomeTeamTotalPayout,BarstoolAwayTeamTotalPayout
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-02,2019,RegularSeason,1/2/19,8:00 PM ET,Wed,1/2/19 @ 8:00 PM ET,NaN,Final,ORL,112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,10:30 PM ET,Wed,1/2/19 @ 10:30 PM ET,NaN,Final,OKC,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,9:00 PM ET,Wed,1/2/19 @ 9:00 PM ET,NaN,Final,PHI,132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,8:00 PM ET,Wed,1/2/19 @ 8:00 PM ET,NaN,Final,DET,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,8:00 PM ET,Wed,1/2/19 @ 8:00 PM ET,NaN,Final,MIN,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
odds_f.drop(datetime(2019, 2, 17), axis=0, inplace=True)
odds_f.drop(datetime(2022, 2, 20), axis=0, inplace=True)
odds_f.drop(datetime(2020, 2, 16), axis=0, inplace=True)
odds_f.drop(datetime(2021, 3, 7), axis=0, inplace=True)

In [38]:
odds_f_sub = odds_f[["AwayTeam", "HomeTeam","HomeTeamHasWon","ConsensusHomeMoneyLine", "ConsensusAwayMoneyLine"]].copy()

In [39]:
odds_f_sub.head()

,AwayTeam,HomeTeam,HomeTeamHasWon,ConsensusHomeMoneyLine,ConsensusAwayMoneyLine
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0


In [40]:
pre_f.head()

,Unnamed: 0,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win,date,winprob_home,winprob_away
0,0,Philadelphia 76ers,4,2,2,0.500,48.0,34.5,79.5,0.434,...,27,26,12,22,16,17,0,2007-11-09,0.882803,0.117198
1,1,Charlotte Bobcats,4,2,2,0.500,48.0,30.5,75.5,0.404,...,1,13,29,11,9,16,1,2007-11-09,0.287085,0.712915
2,2,Boston Celtics,3,3,0,1.000,49.7,40.7,78.0,0.521,...,5,30,9,19,21,14,1,2007-11-09,0.929955,0.070045
3,3,New York Knicks,3,2,1,0.667,48.0,40.3,82.3,0.490,...,22,7,6,6,13,9,0,2007-11-09,0.332229,0.667771
4,4,Miami Heat,4,0,4,0.000,48.0,33.3,79.5,0.418,...,23,9,1,21,10,11,0,2007-11-09,0.217731,0.782269


In [41]:
pre_f.loc[:, "datetime"]  = pre_f.loc[:, "date"]

In [42]:
pre_f.head()

,Unnamed: 0,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win,date,winprob_home,winprob_away,datetime
0,0,Philadelphia 76ers,4,2,2,0.500,48.0,34.5,79.5,0.434,...,26,12,22,16,17,0,2007-11-09,0.882803,0.117198,2007-11-09
1,1,Charlotte Bobcats,4,2,2,0.500,48.0,30.5,75.5,0.404,...,13,29,11,9,16,1,2007-11-09,0.287085,0.712915,2007-11-09
2,2,Boston Celtics,3,3,0,1.000,49.7,40.7,78.0,0.521,...,30,9,19,21,14,1,2007-11-09,0.929955,0.070045,2007-11-09
3,3,New York Knicks,3,2,1,0.667,48.0,40.3,82.3,0.490,...,7,6,6,13,9,0,2007-11-09,0.332229,0.667771,2007-11-09
4,4,Miami Heat,4,0,4,0.000,48.0,33.3,79.5,0.418,...,9,1,21,10,11,0,2007-11-09,0.217731,0.782269,2007-11-09


In [43]:
pre_f.rename(columns={"winprob_away": "home_predict_winprob",
                   "winprob_home": "away_predict_winprob",
                   "TEAM_NAME": "home_team",
                   "TEAM_NAME.1": "away_team"}, inplace=True)

In [44]:
odds_f_sub.columns

Index(['AwayTeam', 'HomeTeam', 'HomeTeamHasWon', 'ConsensusHomeMoneyLine',
       'ConsensusAwayMoneyLine'],
      dtype='object')

In [45]:
odds_f_sub.rename(columns = {
    'AwayTeam':'away_team',
    'HomeTeam':'home_team',
    'HomeTeamHasWon':'home_win',
    'ConsensusHomeMoneyLine': 'money_line_home',
    'ConsensusAwayMoneyLine': 'money_line_away'
}, inplace=True)

In [46]:
odds_f_sub.head()

,away_team,home_team,home_win,money_line_home,money_line_away
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0


In [47]:
odds_f_sub["away_team"].unique()

array(['ORL', 'OKC', 'PHI', 'DET', 'MIN', 'NO', 'DAL', 'ATL', 'MIA',
       'HOU', 'DEN', 'TOR', 'UTA', 'IND', 'WAS', 'BKN', 'LAC', 'NY', 'GS',
       'CHA', 'MEM', 'LAL', 'SA', 'SAC', 'CLE', 'MIL', 'CHI', 'PHO',
       'BOS', 'POR'], dtype=object)

In [48]:
pre_f["away_team"].unique()

array(['Toronto Raptors', 'Indiana Pacers', 'Atlanta Hawks',
       'Orlando Magic', 'Phoenix Suns', 'Denver Nuggets',
       'Los Angeles Clippers', 'San Antonio Spurs', 'Milwaukee Bucks',
       'Memphis Grizzlies', 'Minnesota Timberwolves',
       'Cleveland Cavaliers', 'Utah Jazz', 'Boston Celtics',
       'Dallas Mavericks', 'Washington Wizards', 'Miami Heat',
       'New Orleans Hornets', 'Houston Rockets', 'Detroit Pistons',
       'Sacramento Kings', 'Seattle SuperSonics', 'Philadelphia 76ers',
       'Los Angeles Lakers', 'New York Knicks', 'Charlotte Bobcats',
       'New Jersey Nets', 'Portland Trail Blazers', 'Chicago Bulls',
       'Golden State Warriors', 'Oklahoma City Thunder', 'Brooklyn Nets',
       'New Orleans Pelicans', 'Charlotte Hornets', 'LA Clippers'],
      dtype=object)

In [49]:
for x in odds_f_sub["away_team"].unique():
    if x not in TEAM_NAME_MAPPING.keys():
        print(x)

ORL
OKC
PHI
DET
MIN
NO
DAL
ATL
MIA
HOU
DEN
TOR
UTA
IND
WAS
BKN
LAC
NY
GS
CHA
MEM
LAL
SA
SAC
CLE
MIL
CHI
PHO
BOS
POR


In [50]:
print()
TEAM_NAME_MAPPING_REVERT = {}
for key, value in TEAM_NAME_MAPPING.items():
   if value in new_dict:
       TEAM_NAME_MAPPING_REVERT[value] = key
   else:
       TEAM_NAME_MAPPING_REVERT[value]=key
  

NameError: name 'new_dict' is not defined

In [51]:
TEAM_NAME_MAPPING = {
 'Atlanta Hawks': 'ATL',
 'Brooklyn Nets': 'BKN',
 'Boston Celtics': 'BOS',
 'Charlotte Hornets': 'CHA',
 'Chicago Bulls': 'CHI',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Golden State Warriors': 'GS',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'Los Angeles Clippers': 'LAC',
 'Los Angeles Lakers': 'LAL',
 'Memphis Grizzlies': 'MEM',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NO',
 'New York Knicks': 'NY',
 'Oklahoma City Thunder': 'OKC',
 'Orlando Magic': 'ORL',
 'Philadelphia 76ers': 'PHI',
 'Phoenix Suns': 'PHO',
 'Portland Trail Blazers': 'POR',
 'Sacramento Kings': 'SAC',
 'San Antonio Spurs': 'SA',
 'Toronto Raptors': 'TOR',
 'Utah Jazz': 'UTA',
 'Washington Wizards': 'WAS',
 'New Jersey Nets': 'NKN',
 'Seattle SuperSonics': 'SS',
 'Charlotte Bobcats': 'CB',
 'New Orleans Hornets': 'NOH',
 'LA Clippers': 'LC',
}

In [52]:
len(pre_f["away_team"].unique())

35

In [53]:
pre_f["away_team"] = pre_f["away_team"].apply(lambda x: TEAM_NAME_MAPPING[x])
pre_f["home_team"] = pre_f["home_team"].apply(lambda x: TEAM_NAME_MAPPING[x])

In [54]:
pre_f["away_team"].unique()

array(['TOR', 'IND', 'ATL', 'ORL', 'PHO', 'DEN', 'LAC', 'SA', 'MIL',
       'MEM', 'MIN', 'CLE', 'UTA', 'BOS', 'DAL', 'WAS', 'MIA', 'NOH',
       'HOU', 'DET', 'SAC', 'SS', 'PHI', 'LAL', 'NY', 'CB', 'NKN', 'POR',
       'CHI', 'GS', 'OKC', 'BKN', 'NO', 'CHA', 'LC'], dtype=object)

In [55]:
odds_f_sub

,away_team,home_team,home_win,money_line_home,money_line_away
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0
...,...,...,...,...,...
2022-03-27,PHI,PHO,True,-185.0,152.0
2022-03-27,GS,WAS,True,226.0,-281.0
2022-03-27,LAL,NO,True,-173.0,143.0


In [56]:
odds_f_sub.loc[:, "key"] = odds_f_sub.index.astype(str) + odds_f_sub["home_team"] + odds_f_sub["away_team"]
pre_f.loc[:, "key"] = pre_f["datetime"].astype(str) + pre_f["home_team"] + pre_f["away_team"]

In [57]:
odds_f_sub.set_index("key", inplace = True)
pre_f.set_index("key", inplace = True)

In [58]:
odds_f_sub.merge(pre_f, left_index=True, right_index=True, how='inner').to_csv("data.csv")

In [59]:
combine_f = odds_f_sub.merge(pre_f, left_index=True, right_index=True, how='inner')

In [60]:
(combine_f.home_team_x == combine_f.home_team_y).all()

True